In [23]:
!pip install matplotlib pandas yfinance pandas_datareader investpy tabulate ta


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=7de3aad51cefd4c5fce5a2e47f755baf4918d2c1f59f4127d7044d339f4199db
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


normalizar os preços de fechamento de forma que o valor inicial de cada ativo seja ajustado para 1 e os subsequentes serão proporcionais a esse valor inicial.

Esse tipo de normalização é chamado de base 1 (ou normalização relativa), onde o primeiro valor de fechamento é definido como 1 e todos os valores seguintes são calculados com base nesse valor. A fórmula para isso é:

Pre
c
¸
o Normalizado
𝑖
=
Pre
c
¸
o de Fechamento
𝑖
Pre
c
¸
o de Fechamento
0
Pre
c
¸
​
 o Normalizado
i
​
 =
Pre
c
¸
​
 o de Fechamento
0
​

Pre
c
¸
​
 o de Fechamento
i
​

​

Onde:

Pre
c
¸
o de Fechamento
𝑖
Pre
c
¸
​
 o de Fechamento
i
​
  é o preço de fechamento do ativo no dia
𝑖
i,
Pre
c
¸
o de Fechamento
0
Pre
c
¸
​
 o de Fechamento
0
​
  é o preço de fechamento do ativo no primeiro dia de dados disponíveis.

Para calcular a variação percentual anualizada de um ativo, podemos usar a fórmula da taxa de crescimento anual composta (CAGR, do inglês Compound Annual Growth Rate), que é a taxa de retorno constante que teria sido necessária para que o valor inicial de um ativo se transformasse no valor final, considerando o número de anos do período.

A fórmula para o CAGR é:

𝐶
𝐴
𝐺
𝑅
=
(
𝑉
𝑎
𝑙
𝑜
𝑟

𝐹
𝑖
𝑛
𝑎
𝑙
𝑉
𝑎
𝑙
𝑜
𝑟

𝐼
𝑛
𝑖
𝑐
𝑖
𝑎
𝑙
)
1
𝑛
−
1
CAGR=(
Valor Inicial
Valor Final
​
 )
n
1
​

 −1
Onde:

Valor Final é o preço de fechamento do último dia.
Valor Inicial é o preço de fechamento do primeiro dia.
n é o número de anos no período.
Vamos atualizar a função relatorio para calcular a variação percentual anualizada (CAGR) de cada ativo.

In [58]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator  # Para cálculo do RSI
from tabulate import tabulate  # Para visualização do DataFrame no console

def calcular_rsi(df: pd.DataFrame, coluna: str, period: int) -> pd.Series:
    """
    Calcula o Índice de Força Relativa (RSI) baseado em uma coluna específica do DataFrame.
    """
    try:
        if coluna not in df.columns or df.empty:
            raise ValueError("Coluna inexistente ou DataFrame vazio.")
        rsi = RSIIndicator(df[coluna], window=period)
        return rsi.rsi()
    except Exception as e:
        print(f"Erro ao calcular RSI: {e}")
        return pd.Series([None] * len(df), index=df.index)

def calc_dy_medio(df):
    """
    Calcula o Dividend Yield médio dos ativos.
    """
    if 'DY (%)' in df.columns and not df.empty:
        dy_medio = df['DY (%)'].mean()
        return round(dy_medio, 2)
    else:
        return 0

def alocacao_ativos(df, dy_medio):
    """
    Ajusta a distribuição dos ativos para que o DY ponderado atinja ou ultrapasse
    o DY médio + 2%.
    """
    if 'DY (%)' in df.columns and not df.empty:
        dy_alvo = dy_medio + 2

        # Inicializar distribuição igualitária
        df['Distribuição'] = 1 / len(df)

        # Ordenar ativos por DY em ordem decrescente
        df = df.sort_values(by='DY (%)', ascending=False).reset_index(drop=True)

        # Ajustar a distribuição para atingir o DY alvo
        restante = 1.0  # Proporção restante para distribuir
        dy_ponderado = 0

        for i, row in df.iterrows():
            if dy_ponderado >= dy_alvo:
                break

            proporcao_max = min(restante, 0.5)  # Limitar a no máximo 50% por ativo
            df.at[i, 'Distribuição'] = proporcao_max
            restante -= proporcao_max

            dy_ponderado = (df['DY (%)'] * df['Distribuição']).sum()

        # Reajustar qualquer proporção restante nos ativos de maior DY
        if dy_ponderado < dy_alvo and restante > 0:
            for i, row in df.iterrows():
                adicional = min(restante, 0.5 - df.at[i, 'Distribuição'])
                df.at[i, 'Distribuição'] += adicional
                restante -= adicional
                dy_ponderado = (df['DY (%)'] * df['Distribuição']).sum()
                if dy_ponderado >= dy_alvo:
                    break

        # Normalizar a distribuição para garantir soma igual a 1
        df['Distribuição'] /= df['Distribuição'].sum()

        return round(dy_ponderado, 2), df
    else:
        return 0, df

def verificar_compra(dy, rsi):
    """
    Indica se é uma boa compra (se o Dividend Yield >= 10% e RSI está entre 25 e 50).
    """
    return "COMPRA" if dy >= 10 or 25 <= rsi <= 50 else "VENDA"

def show_data(ativos):
    """
    Obtém os dados de cada ativo e os armazena em uma lista de dicionários.
    """
    dados_ativos = []
    for ativo in ativos:
        try:
            ticker = yf.Ticker(ativo)
            info = ticker.info
            dados = ticker.history(period="30d", interval="1d")

            if dados.empty:
                print(f"Não foi possível obter dados históricos para o ativo {ativo}.")
                continue

            df = dados[['Close']].reset_index()
            df.columns = ['Data', 'Fechamento']

            nome = info.get('shortName', 'N/A')
            dy = info.get('dividendYield', 0)
            dy = dy * 100 if dy is not None else 0
            price = info.get('currentPrice', 0)
            max52wk = info.get('fiftyTwoWeekHigh', 0)

            rsi = calcular_rsi(df, coluna='Fechamento', period=7).iloc[-1]

            dados_ativos.append({
                'Ativo': ativo,
                'Nome': nome,
                'DY (%)': round(dy, 2),
                'Preço (R$)': round(price, 2),
                'Máxima (R$)': round(max52wk, 2),
                'RSI': round(rsi, 2) if pd.notna(rsi) else None,
                'Status': verificar_compra(dy, rsi if pd.notna(rsi) else 0),
            })
        except Exception as e:
            print(f"Erro ao obter dados para {ativo}: {e}")
    return dados_ativos

def main():
    ativos = [
        'BBDC3.SA', 'BBAS3.SA', 'BBSE3.SA', 'CMIG4.SA', 'TAEE4.SA',
        'VALE3.SA', 'ROXO34.SA', 'KNCR11.SA', 'XPML11.SA',
        'VISC11.SA', 'BTLG11.SA', 'MCCI11.SA',
    ]

    dados_ativos = show_data(ativos)
    df = pd.DataFrame(dados_ativos)

    if not df.empty:
        print("\nVisualização do DataFrame:")
        def color_status(val):
            if val == "COMPRA":
                return f"\033[92m{val}\033[0m"  # Verde
            elif val == "VENDA":
                return f"\033[91m{val}\033[0m"  # Vermelho
            return val

        df['Status'] = df['Status'].apply(color_status)
        print(tabulate(df.sort_values(by='DY (%)', ascending=False), headers='keys', tablefmt='psql', showindex=False))

        dy_medio = calc_dy_medio(df)
        dy_ponderado, df = alocacao_ativos(df, dy_medio)

        print("\nCARTEIRA:")
        print(f"\nQuantidade: {len(ativos)} ativos")
        print(f"DY médio: {dy_medio}%")
        print(f"DY ponderado ajustado: {dy_ponderado}%")

if __name__ == "__main__":
    main()



Visualização do DataFrame:
+-----------+------------------------+----------+--------------+---------------+-------+----------+
| Ativo     | Nome                   |   DY (%) |   Preço (R$) |   Máxima (R$) |   RSI | Status   |
|-----------+------------------------+----------+--------------+---------------+-------+----------|
| VALE3.SA  | VALE        ON      NM |    11.54 |        53.03 |         69.6  | 49.01 | COMPRA   |
| MCCI11.SA | FII MAUA    CI  ER     |    11.54 |        73.68 |         95.8  | 26.23 | COMPRA   |
| KNCR11.SA | FII KINEA RICI         |    11.32 |       101.8  |        107.36 | 46.86 | COMPRA   |
| VISC11.SA | FII VINCI SCCI         |     9.41 |        94.1  |        124.4  | 49.54 | COMPRA   |
| XPML11.SA | FII XP MALLSCI         |     9.13 |        95.01 |        119.1  | 38.94 | COMPRA   |
| BTLG11.SA | FII BTLG    CI  ER     |     9.12 |        91.81 |        108.48 | 43.66 | COMPRA   |
| BBAS3.SA  | BRASIL      ON      NM |     8    |        26.62 |        